In [1]:
debug = False

import ROOT as R
if debug:
    R.ROOT.EnableImplicitMT() # only for ROOT rdf

import os
from pprint import pprint
import pandas as pd 


Welcome to JupyROOT 6.19/01


In [2]:
import PyRDF
if not debug:
    PyRDF.use("spark")
PyRDF.include_headers("headers.hh")

print(PyRDF.current_backend.supported_operations)

['Define', 'Filter', 'Histo1D', 'Histo2D', 'Histo3D', 'Profile1D', 'Profile2D', 'Profile3D', 'Count', 'Sum', 'Fill', 'Snapshot', 'Graph', 'AsNumpy']


In [3]:
# this is only for root rdf "interactive"
if debug:
    R.gInterpreter.ProcessLine(".L headers.hh")

In [4]:
import latinos_rdf as lrdf

In [5]:
#samples = ["DATA","VBS", "VBF-V", "Wjets", "singleTop", "ttbar",  "VV","DY","FakeQCD", "VVV"]
samples = ["VBS", "Wjets", "DY", "top"]
weights = ["weight", "XSWeight"]
cut = "lowen_looseVBS"
#cut = "boost_looseVBS"
output_basedir = "/eos/user/d/dmapelli/public/latino/Full2017v6_200121_v0/"
output_dir = output_basedir + cut + "/"
config_dir = "Full2017v6"

import os 
os.makedirs(output_dir, exist_ok=True)

In [6]:
config = lrdf.ConfigReader(config_dir)

columns_MC = config.variables

#columns_DATA = columns
columns_MC = [ cut+"_"+c for c in columns_MC]
columns_MC = weights + columns_MC

In [7]:
joblist = []

for sample in samples:
    if debug:
        trees, nfiles = lrdf.build_dataframe(config_dir, sample, R, "root") # ROOT RDF "interactive"
    else:
        trees, nfiles = lrdf.build_dataframe(config_dir,  sample, PyRDF , "pyrdf") # Spark
    for tree, nfile in zip(trees,nfiles):
        joblist.append((tree,nfile))
        
######## Now sort by number of files
jobslist = sorted(joblist, key=lambda v: v[1], reverse=True)


In [ ]:
########
dfs = {}
for tree,_ in jobslist:
        # print(tree)
        # Choose the right output columns
        if sample == "DATA": outputcols = columns_DATA
        else: outputcols = columns_MC

        df = pd.DataFrame(tree[cut].rdf_node.AsNumpy(columns=outputcols))
        print(tree.name, df.shape)
        
        if tree.name in dfs:
            df.to_pickle(output_dir + tree.name + f"_part{dfs[tree.name]+1}.pkl")
            dfs[tree.name] +=1
        else:
            df.to_pickle(output_dir + tree.name +"_part1.pkl")
            dfs[tree.name] =1
            
        

Wjets (605804, 56)
top (52217, 56)
